<a href="https://colab.research.google.com/github/LikhanInSpace/bomFatabo/blob/main/vgg16_model_for_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
from __future__ import print_function
!pip install keras_applications



import numpy as np
import warnings
from keras.models import Model
from keras.layers import  Flatten, Dense, Dropout
from keras.layers import  Input, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs

In [229]:
def VGG16(input_tensor=None, classes=2):
  img_rows, img_cols= 224,224
  img_channels= 3

  img_dim=(img_rows,img_cols,img_channels)
  img_input = Input(shape=img_dim)

  # block 1
  x = Convolution2D(64, (3,3), activation='relu', padding='same', name='block1_conv1')(img_input)
  x = Convolution2D(64, (3,3), activation='relu', padding='same', name='block1_conv2')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block1_pool')(x)

  # block 2
  x = Convolution2D(128, (3,3), activation='relu', padding='same', name='block2_conv1')(x)
  x = Convolution2D(128, (3,3), activation='relu', padding='same', name='block2_conv2')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block2_pool')(x)

  # block 3
  x = Convolution2D(256, (3,3), activation='relu', padding='same', name='block3_conv1')(x)
  x = Convolution2D(256, (3,3), activation='relu', padding='same', name='block3_conv2')(x)
  x = Convolution2D(256, (3,3), activation='relu', padding='same', name='block3_conv3')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block3_pool')(x)

  # block 4
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block4_conv1')(x)
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block4_conv2')(x)
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block4_conv3')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block4_pool')(x)

  # block 5
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block5_conv1')(x)
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block5_conv2')(x)
  x = Convolution2D(512, (3,3), activation='relu', padding='same', name='block5_conv3')(x)
  x = MaxPooling2D((2,2), strides=(2,2), name='block5_pool')(x)

  # classification block
  x = Flatten(name='flatten')(x)
  x = Dense(4096, activation='relu', name= 'fc1')(x)
  x = Dense(4096, activation='relu', name= 'fc2')(x)
  x = Dense(classes, activation='softmax', name='predictions')(x)

  # create model

  model = Model(inputs = img_input, outputs = x, name='vgg16')
  return model






In [228]:
#new section

model = VGG16(classes = 2) # for real and fake image
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [230]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [231]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [239]:
dataset_path = os.listdir('/content/drive/MyDrive/thesis dataset/type of image')
image_types = os.listdir('/content/drive/MyDrive/thesis dataset/type of image')
#label_types = os.listdir('/content/drive/MyDrive/thesis dataset/type of image')
print (image_types) # what kinds image in this dataset
print("Types of image found:", len(dataset_path))

['so called fake', 'so called real']
Types of image found: 2


In [240]:
images = []
for item in image_types:
  all_images = os.listdir('/content/drive/MyDrive/thesis dataset/type of image/' + '/' + item)
   
for image in all_images:
  images.append((item, str('/content/drive/MyDrive/thesis dataset/type of image/'+ '/' + item) + '/' + image))

#labels = []
#for item1 in label_types:
 # all_labels = os.listdir('/content/drive/MyDrive/thesis dataset/type of image/' + '/' + item1)
   
#for label in all_labels:
  #labels.append((item1, str('/content/drive/MyDrive/thesis dataset/type of image/'+ '/' + item1) + '/' + label))
  print(images)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [241]:
# build a dataframe
images_df = pd.DataFrame(data=images, columns=['image type','image'])
print(images_df.head())

       image type                                              image
0  so called real  /content/drive/MyDrive/thesis dataset/type of ...
1  so called real  /content/drive/MyDrive/thesis dataset/type of ...
2  so called real  /content/drive/MyDrive/thesis dataset/type of ...
3  so called real  /content/drive/MyDrive/thesis dataset/type of ...
4  so called real  /content/drive/MyDrive/thesis dataset/type of ...


In [242]:
print("Total number of images in the dataset:", len(images_df))
image_count = images_df['image type'].value_counts()
print("Images in each category:")
print(image_count)

Total number of images in the dataset: 303
Images in each category:
so called real    303
Name: image type, dtype: int64


In [245]:
import cv2
path = '/content/drive/MyDrive/thesis dataset/type of image/'

im_size=224

images = []
labels = []

for i in image_types:
  data_path = path + str(i)
  filenames = [i for i in os.listdir(data_path)]

  for f in filenames:
    img = cv2.imread(data_path + '/' + f)
    img = cv2.resize(img, (im_size, im_size))
    images.append(img)
    labels.append(i)
  
  print(labels)
  print(images)


Streaming output truncated to the last 5000 lines.

       [[168, 184, 203],
        [168, 184, 203],
        [168, 184, 203],
        ...,
        [168, 179, 190],
        [173, 184, 196],
        [174, 186, 198]]], dtype=uint8), array([[[129, 141, 160],
        [129, 141, 160],
        [129, 141, 160],
        ...,
        [ 32,  46,  67],
        [ 33,  47,  68],
        [ 33,  48,  69]],

       [[128, 140, 160],
        [128, 140, 160],
        [128, 140, 160],
        ...,
        [ 31,  45,  65],
        [ 31,  46,  66],
        [ 31,  46,  67]],

       [[128, 140, 160],
        [128, 140, 160],
        [128, 140, 160],
        ...,
        [ 28,  42,  61],
        [ 28,  43,  62],
        [ 28,  43,  62]],

       ...,

       [[171, 182, 202],
        [171, 182, 202],
        [171, 182, 202],
        ...,
        [169, 180, 191],
        [173, 185, 197],
        [174, 186, 198]],

       [[171, 183, 202],
        [171, 183, 202],
        [171, 183, 202],
        ...,
        

In [246]:
images = np.array(images)
images = images.astype('float32')/255.0
images.shape

(606, 224, 224, 3)

In [247]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
y = images_df['image type'].values

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform(y)

print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [248]:
y= y.reshape(-1,1)
ct = ColumnTransformer([('image type', OneHotEncoder(), [0])], remainder='passthrough')
y = ct.fit_transform(y) #.toarray()

Y= ct.fit_transform(y)
Y.shape

(303, 1)

In [249]:
len(images)==len(labels)

True

In [250]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np


images, Y= shuffle(images, Y , random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

train_x=np.array(train_x)
test_x=np.array(test_x)
train_y=np.array(train_y)
test_y=np.array(test_y)

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

ValueError: ignored

In [227]:
model.fit(train_x, train_y, epochs=10, batch_size=32)

Epoch 1/10


ValueError: ignored